In [0]:
# %load_ext autoreload
# %autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import mlflow
import pandas as pd
from typing import Dict, List, Any, Optional, Generator
import json
import os
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput
from databricks.sdk.service.serving import DataframeSplitInput

from mlflow.pyfunc import ChatAgent

from mlflow.types.agent import (
    ChatAgentMessage,
    ChatAgentResponse,
    ChatAgentChunk,
    ChatContext
)

from databricks import agents
from mlflow.models import ModelConfig
from mlflow.models.resources import DatabricksServingEndpoint

import uuid  # Add at top of file
from uuid import UUID

In [0]:
%pip install databricks-agents databricks-sdk mlflow databricks-sdk

In [0]:
%%writefile invoice_agent.py

import mlflow
import mlflow.pyfunc
import pandas as pd
from typing import Dict, List, Any, Optional, Generator
import json
import os
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput
from databricks.sdk.service.serving import DataframeSplitInput

from mlflow.pyfunc import ChatAgent

from mlflow.types.agent import (
    ChatAgentMessage,
    ChatAgentResponse,
    ChatAgentChunk,
    ChatContext
)

from databricks import agents
from mlflow.models import ModelConfig
from mlflow.models.resources import DatabricksServingEndpoint

import uuid  # Add at top of file
from uuid import UUID

class InvoiceProcessingAgent(ChatAgent):
# class InvoiceProcessingAgent(mlflow.pyfunc.PythonModel):
    
    def __init__(self, endpoint_name: str = "databricks-claude-3-7-sonnet"):
        """
        Initialize the Invoice Processing Agent.
        
        Args:
            endpoint_name: Name of the Databricks Foundation Model endpoint to use
        """
        self.endpoint_name = endpoint_name
        self.workspace_client = WorkspaceClient()
        self.prompts = self._load_static_prompts()
        
        # Initialize OpenAI client for the foundation model
        self.openai_client = self.workspace_client.serving_endpoints.get_open_ai_client()
        
        # Configure agent parameters
        self.config = ModelConfig(development_config={
            "endpoint_name": endpoint_name,
            "temperature": 0.0,
            "max_tokens": 4096
        })
    
    def _load_static_prompts(self) -> Dict[str, str]:
        """Load the prompts from JSON"""
        prompts = {
            "document_classification": "Classify this document into one of the following categories: invoice, receipt, ID, contract.",
            "information_extraction": "Extract the following fields from this document: date, total amount, vendor name, transaction ID."
        }
        return prompts
    
    def predict(
        self,
        messages: List[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[Dict[str, Any]] = None
    ) -> ChatAgentResponse:
        """
        Process a user query and return a response according to ChatAgent interface.
        
        Args:
            messages: List of ChatAgentMessage objects
            context: Optional ChatContext object
            custom_inputs: Optional additional inputs
            
        Returns:
            ChatAgentResponse containing the agent's response
        """
        # Add unique ID
        message_id = str(uuid.uuid4())

        # Extract document text from the user message
        user_messages = [msg for msg in messages if msg.role == "user"]
        if not user_messages:
            return ChatAgentResponse(messages=[
                ChatAgentMessage(id=message_id, role="assistant", content="Please provide a document to process.")
            ])
        
        document_text = user_messages[-1].content
        
        # Process the document
        document_class = self._classify_document(document_text)
        extracted_info = self._extract_information(document_text)
        
        # Format the response
        result = {
            "document_class": document_class,
            "extracted_information": extracted_info
        }
        
        # Create and return the agent response
        return ChatAgentResponse(messages=[
            ChatAgentMessage(
                id=message_id,
                role="assistant", 
                content=f"Document classified as: {document_class}\n\nExtracted information: {json.dumps(extracted_info, indent=2)}"
            )
        ])
    
    def predict_stream(
        self,
        messages: List[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[Dict[str, Any]] = None
    ) -> Generator[ChatAgentChunk, None, None]:
        """
        Stream the response for a user query according to ChatAgent interface.
        
        Args:
            messages: List of ChatAgentMessage objects
            context: Optional ChatContext object
            custom_inputs: Optional additional inputs
            
        Yields:
            ChatAgentChunk objects containing parts of the response
        """
        message_id = str(uuid.uuid4())

        # Extract document text from the user message
        user_messages = [msg for msg in messages if msg.role == "user"]
        if not user_messages:
            yield ChatAgentChunk(delta=ChatAgentMessage(
                id=message_id, 
                role="assistant", 
                content="Please provide a document to process."
            ))
            return
        
        document_text = user_messages[-1].content
        
        # Process the document (in real implementation, this would be streamed)
        document_class = self._classify_document(document_text)
        
        # Yield the classification result
        yield ChatAgentChunk(delta=ChatAgentMessage(
            id=message_id, 
            role="assistant", 
            content=f"Document classified as: {document_class}\n\n"
        ))
        
        # Process and yield the extraction result
        extracted_info = self._extract_information(document_text)
        yield ChatAgentChunk(delta=ChatAgentMessage(
            id=message_id, 
            role="assistant", 
            content=f"Extracted information: {json.dumps(extracted_info, indent=2)}"
        ))
    
    def _classify_document(self, document_text: str) -> str:
        """
        Classify the document using foundation model.
        """
        classification_prompt = self.prompts["document_classification"]
        
        response = self.openai_client.chat.completions.create(
            model=self.endpoint_name,
            messages=[
                {"role": "system", "content": "You are an expert document classifier."},
                {"role": "user", "content": f"{classification_prompt}\n\nDocument: {document_text}"}
            ],
            temperature=0.0,
            max_tokens=256
        )
        
        return response.choices[0].message.content.strip()
    
    def _extract_information(self, document_text: str) -> Dict[str, Any]:
        """
        Extract information from the document using foundation model.
        """
        extraction_prompt = self.prompts["information_extraction"]
        
        response = self.openai_client.chat.completions.create(
            model=self.endpoint_name,
            messages=[
                {"role": "system", "content": "You are an expert at extracting structured information from documents. Return the information in JSON format."},
                {"role": "user", "content": f"{extraction_prompt}\n\nDocument: {document_text}"}
            ],
            temperature=0.0,
            max_tokens=1024
        )
        
        try:
            # Try to parse as JSON
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            # If not valid JSON, return as text
            return {"raw_extraction": response.choices[0].message.content.strip()}


AGENT = InvoiceProcessingAgent()

mlflow.models.set_model(AGENT)

In [0]:
import mlflow
# from getting_started_agent import LLM_ENDPOINT, baseline_config, tools
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool
from invoice_agent import InvoiceProcessingAgent

# TODO fill in your catalog and schema name
# catalog = "demos"
# schema = "z_agents_002"

# TODO: Replace with your model serving endpoint
LLM_ENDPOINT = "databricks-claude-3-7-sonnet"

baseline_config = {
    "endpoint_name": LLM_ENDPOINT,
    "temperature": 0.00,
    "max_tokens": 4096,
    "system_prompt": "You are an expert document classifier and expert at extracting structured information from documents. Return the information in JSON format."
}

resources = [DatabricksServingEndpoint(endpoint_name='databricks-claude-3-7-sonnet')]

tools = []

for tool in tools:
    if isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        python_model="invoice_agent.py",
        artifact_path="invoice_agent",
        model_config=baseline_config,
        resources=resources,
        pip_requirements=[
            "mlflow",
            "databricks-agents",
            "databricks-sdk",
            "databricks-openai",
            "langchain",
            "langgraph==0.3.4",
            "databricks-langchain",
            "unitycatalog-langchain[databricks]",
            "pydantic",
        ],
        input_example={
            "messages": [{"role": "user", "content": "INVOICE #123\nDate: 2024-01-01\nTotal: $100.00"}]
        },
    )

In [0]:
import mlflow
from databricks import agents

# Connect to the Unity catalog model registry
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog and schema for your UC model
catalog = "demos2025"
schema = "source_files"

assert (catalog and schema)

UC_MODEL_NAME = f"{catalog}.{schema}.invoice_agent"

# Register to Unity catalog
uc_registered_model_info = mlflow.register_model(
    model_uri=model_info.model_uri, name=UC_MODEL_NAME
)

# Deploy to enable the review app and create an API endpoint
deployment_info = agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version)

In [0]:
# Example usage in a Databricks notebook
from invoice_agent import InvoiceProcessingAgent

def main():
    # Create an instance of the agent for direct usage
    agent = InvoiceProcessingAgent(endpoint_name="databricks-claude-3-7-sonnet")
    
    # Example document text
    document_text = """
    INVOICE
    
    Invoice #: INV-2023-1234
    Date: 2023-11-15
    
    Vendor: ABC Office Supplies
    Customer: XYZ Corporation
    
    Items:
    - Office paper (10 reams): $45.00
    - Printer ink cartridges (2): $65.00
    - Desk organizers (5): $25.00
    
    Subtotal: $135.00
    Tax (10%): $13.50
    Total: $148.50
    
    Payment due within 30 days.
    """
    
    # Process the document using ChatAgent interface
    result = agent.predict([
        ChatAgentMessage(role="user", content=document_text)
    ])
    
    # Print the result
    print(result.messages[0].content)
    
    # Deploy the agent if needed
    # deployment = deploy_agent()

In [0]:
main()